In [39]:
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random
import time
import pickle
from sklearn.preprocessing import MinMaxScaler

In [40]:
# Đọc dữ liệu từ dataset
train_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Train.csv'
test_url = 'https://raw.githubusercontent.com/merteroglu/NSL-KDD-Network-Instrusion-Detection/master/NSL_KDD_Test.csv'

col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

df = pd.read_csv(train_url,header=None, names = col_names)

df_test = pd.read_csv(test_url, header=None, names = col_names)

print("Dimensions of Training set: ", df.shape)
print("Dimensions of Test set: ", df_test.shape)

Dimensions of Training set:  (125973, 42)
Dimensions of Test set:  (22544, 42)


In [41]:
# Xử lí dữ liệu trên 3 cột đặc biệt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']

df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()

# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
print(unique_protocol2)

# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
print(unique_service2)


# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
print(unique_flag2)


# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2


#do it for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2

print(dumcols)
print(testdumcols)

['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp']
['service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames', 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 'service_remote_job', 'ser

In [42]:
# convert data on 3 columns to number

# Train set
df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)

# test
print(df_categorical_values.head())
print('--------------------')
print(df_categorical_values_enc.head())

# Test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)

  protocol_type   service flag
0           tcp  ftp_data   SF
1           udp     other   SF
2           tcp   private   S0
3           tcp      http   SF
4           tcp      http   SF
--------------------
   protocol_type  service  flag
0              1       20     9
1              2       44     9
2              1       49     5
3              1       24     9
4              1       24     9


In [43]:
# One hot encoder
enc = OneHotEncoder(categories='auto')
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)


# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()

,Protocol_type_icmp,Protocol_type_tcp,Protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [44]:
trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]

for col in difference:
    testdf_cat_data[col] = 0

print(df_cat_data.shape)
print(testdf_cat_data.shape)

(125973, 84)
(22544, 84)


In [45]:
newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)

# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)

print(newdf.shape)
print(newdf_test.shape)
display(newdf)
display(newdf_test)

(125973, 123)
(22544, 123)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,491,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,146,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0,232,8153,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,199,420,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125969,8,105,145,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125970,0,2231,384,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
125971,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_harvest,service_aol,service_urh_i,service_red_i,service_http_8001,service_http_2784
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
3,0,20,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
4,1,0,15,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0,794,333,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22540,0,317,938,0,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22541,0,54540,8314,0,0,0,2,0,1,1,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
22542,0,42,42,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0


In [46]:
labeldf=newdf['label']
labeldf_test=newdf_test['label']


# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1,
                            'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1 })
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 1,'nmap' : 1,'portsweep' : 1,'satan' : 1,'mscan' : 1,'saint' : 1
                           ,'ftp_write': 1,'guess_passwd': 1,'imap': 1,'multihop': 1,'phf': 1,'spy': 1,'warezclient': 1,'warezmaster': 1,'sendmail': 1,'named': 1,'snmpgetattack': 1,'snmpguess': 1,'xlock': 1,'xsnoop': 1,'httptunnel': 1,
                           'buffer_overflow': 1,'loadmodule': 1,'perl': 1,'rootkit': 1,'ps': 1,'sqlattack': 1,'xterm': 1})


# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test

In [47]:
# Split dataframes into X & Y
# X Özellikler , Y sonuç değişkenleri
X_Df = newdf.drop(labels='label',axis=1)
Y_Df = newdf.label

# test set
X_Df_test = newdf_test.drop(labels='label',axis=1)
Y_Df_test = newdf_test.label

In [48]:
colNames=list(X_Df)
colNames_test=list(X_Df_test)

In [13]:
print(colNames)
print(colNames_test)

['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp', 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'se

In [49]:
# Chuẩn hóa dữ liệu
scaler_minmax = MinMaxScaler(feature_range=(0, 1))
scaler_minmax.fit(X_Df_test)

MinMaxScaler()

In [50]:
# Voting classifier
voting_classifier = pickle.load(open("/content/drive/MyDrive/Model/voting_modelv1.pickle", "rb"))

In [51]:
# KNN model
knn_model = pickle.load(open("/content/drive/MyDrive/Model/knn_modelv2.pickle", "rb"))

In [52]:
# SVM model
svm_model = pickle.load(open("/content/drive/MyDrive/Model/svm_modelv1.pickle", "rb"))


In [53]:
# NB model
nb_model = pickle.load(open("/content/drive/MyDrive/Model/nb_modelv1.pickle", "rb"))


In [54]:
# DT model
dt_model = pickle.load(open("/content/drive/MyDrive/Model/dt_modelv1.pickle", "rb"))


In [55]:
# LR model
lr_model = pickle.load(open("/content/drive/MyDrive/Model/lr_modelv1.pickle", "rb"))

In [56]:
# RF model
rf_model = pickle.load(open("/content/drive/MyDrive/Model/rf_modelv1.pickle", "rb"))

In [57]:
# QDA model
qda_model = pickle.load(open("/content/drive/MyDrive/Model/qda_modelv1.pickle", "rb"))

In [58]:
# MLP model
mlp_model = pickle.load(open("/content/drive/MyDrive/Model/mlp_modelv1.pickle", "rb"))

In [59]:
# LDA model
lda_model = pickle.load(open("/content/drive/MyDrive/Model/lda_modelv1.pickle", "rb"))

In [60]:
# GB model
gb_model = pickle.load(open("/content/drive/MyDrive/Model/gb_modelv1.pickle", "rb"))

In [61]:
# BAG model
bag_model = pickle.load(open("/content/drive/MyDrive/Model/bag_modelv1.pickle", "rb"))

In [27]:
# Spilit all vector malicious out Dataset
malicious_vector = newdf_test.loc[newdf_test['label'] == 1]
malicious_vector.reset_index(drop=True, inplace=True)
malicious_vector.drop('label', axis=1, inplace=True)
malicious_vector.head()

<ipython-input-27-3ba5973c6e51>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  malicious_vector.drop('label', axis=1, inplace=True)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_S2,flag_S3,flag_SF,flag_SH,service_harvest,service_aol,service_urh_i,service_red_i,service_http_8001,service_http_2784
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,0,20,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0
3,1,0,15,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,0,129,174,0,0,0,0,1,0,0,...,0.0,0.0,1.0,0.0,0,0,0,0,0,0


In [62]:
# Function Evaluate
def Evaluate(proba_list):
  sum = 0
  for proba in proba_list:
    sum += proba[0]
  result = sum / len(proba_list)
  return result

PSO Algorithm


In [65]:
# Genetic Algorithm

# define range value of features
tags = ["duration","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

cols = tags + dumcols

param_range = {
    'duration':[0,8445],
    'src_bytes':[0,1300000],
    'dst_bytes':[0, 1300000],
    'land':[0,1],
    'wrong_fragment': [0, 10],
    'urgent': [0, 10],
    'hot': [0,101],
    'num_failed_logins':[0, 10],
    'logged_in': [0,1],
    'num_compromised': [0,7479],
    'root_shell': [0,1],
    'su_attempted': [0, 10],
    'num_root': [0,7468],
    'num_file_creations': [0, 10],
    'num_shells':[0, 10],
    'num_access_files':[0, 10],
    'num_outbound_cmds': [0,10],
    'is_host_login':[0,1],
    'is_guest_login':[0,1],
    'count':[0,511],
    'srv_count':[0,511],
    'serror_rate':[0,1],
    'srv_serror_rate':[0,1],
    'rerror_rate':[0,1],
    'srv_rerror_rate':[0,1],
    'same_srv_rate':[0,1],
    'diff_srv_rate':[0,1],
    'srv_diff_host_rate':[0,1],
    'dst_host_count':[0,255],
    'dst_host_srv_count':[0,255],
    'dst_host_same_srv_rate':[0,1],
    'dst_host_diff_srv_rate':[0,1],
    'dst_host_same_src_port_rate':[0,1],
    'dst_host_srv_diff_host_rate':[0,1],
    'dst_host_serror_rate':[0,1],
    'dst_host_srv_serror_rate':[0,1],
    'dst_host_rerror_rate':[0,1],
    'dst_host_srv_rerror_rate':[0,1]
}

features_integer = ["duration","src_bytes",
    "dst_bytes","wrong_fragment","urgent","hot","num_failed_logins",
    "num_compromised","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "count","srv_count"]

features_binary = ["land", "logged_in", "root_shell", "is_host_login", "is_guest_login"]

index_integer = [0, 1, 2, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 19, 20, 28, 29]
index_binary = [3, 8, 10, 18]

In [66]:
from functools import partial
import numpy as np



def _obj_wrapper(func, args, kwargs, x):
    return func(x, *args, **kwargs)

def _is_feasible_wrapper(func, x):
    return np.all(func(x)>=0)

def _cons_none_wrapper(x):
    return np.array([0])

def _cons_ieqcons_wrapper(ieqcons, args, kwargs, x):
    return np.array([y(x, *args, **kwargs) for y in ieqcons])

def _cons_f_ieqcons_wrapper(f_ieqcons, args, kwargs, x):
    return np.array(f_ieqcons(x, *args, **kwargs))

def pso(func, lb, ub, ieqcons=[], f_ieqcons=None, args=(), kwargs={},
        swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=100,
        minstep=1e-8, minfunc=1e-8, debug=False, processes=1,
        particle_output=False):

    assert len(lb)==len(ub), 'Lower- and upper-bounds must be the same length'
    assert hasattr(func, '__call__'), 'Invalid function handle'
    lb = np.array(lb)
    ub = np.array(ub)

    assert np.all(ub>=lb), 'All upper-bound values must be greater than lower-bound values'

    vhigh = np.abs(ub - lb)
    vlow = -vhigh

    # Initialize objective function
    obj = partial(_obj_wrapper, func, args, kwargs)

    # Check for constraint function(s) #########################################
    if f_ieqcons is None:
        if not len(ieqcons):
            if debug:
                print('No constraints given.')
            cons = _cons_none_wrapper
        else:
            if debug:
                print('Converting ieqcons to a single constraint function')
            cons = partial(_cons_ieqcons_wrapper, ieqcons, args, kwargs)
    else:
        if debug:
            print('Single constraint function given in f_ieqcons')
        cons = partial(_cons_f_ieqcons_wrapper, f_ieqcons, args, kwargs)
    is_feasible = partial(_is_feasible_wrapper, cons)

    # Initialize the multiprocessing module if necessary
    if processes > 1:
        import multiprocessing
        mp_pool = multiprocessing.Pool(processes)

    # Initialize the particle swarm ############################################
    S = swarmsize
    D = len(lb)  # the number of dimensions each particle has
    x = np.random.rand(S, D)  # particle positions
    v = np.zeros_like(x)  # particle velocities
    p = np.zeros_like(x)  # best particle positions
    fx = np.zeros(S)  # current particle function values
    fs = np.zeros(S, dtype=bool)  # feasibility of each particle
    fp = np.ones(S)*np.inf  # best particle function values
    g = []  # best swarm position
    fg = np.inf  # best swarm position starting value

    # Initialize the particle's position
    x = lb + x*(ub - lb)

    # Calculate objective and constraints for each particle
    if processes > 1:
        fx = np.array(mp_pool.map(obj, x))
        fs = np.array(mp_pool.map(is_feasible, x))
    else:
        for i in range(S):
            fx[i] = obj(x[i, :])
            fs[i] = is_feasible(x[i, :])

    # Store particle's best position (if constraints are satisfied)
    i_update = np.logical_and((fx < fp), fs)
    p[i_update, :] = x[i_update, :].copy()
    fp[i_update] = fx[i_update]

    # Update swarm's best position
    i_min = np.argmin(fp)
    if fp[i_min] < fg:
        fg = fp[i_min]
        g = p[i_min, :].copy()
    else:
        # At the start, there may not be any feasible starting point, so just
        # give it a temporary "best" point since it's likely to change
        g = x[0, :].copy()

    # Initialize the particle's velocity
    v = vlow + np.random.rand(S, D)*(vhigh - vlow)

    # Iterate until termination criterion met ##################################
    it = 1
    while it <= maxiter:
        print('ok')
        rp = np.random.uniform(size=(S, D))
        rg = np.random.uniform(size=(S, D))

        # Update the particles velocities
        v = omega*v + phip*rp*(p - x) + phig*rg*(g - x)
        # Update the particles' positions
        x = x + v
        # Correct for bound violations
        maskl = x < lb
        masku = x > ub
        x = x*(~np.logical_or(maskl, masku)) + lb*maskl + ub*masku
        print(x)

        # Update objectives and constraints
        if processes > 1:
            fx = np.array(mp_pool.map(obj, x))
            fs = np.array(mp_pool.map(is_feasible, x))
        else:
            for i in range(S):
                fx[i] = obj(x[i, :])
                fs[i] = is_feasible(x[i, :])

        # Store particle's best position (if constraints are satisfied)
        i_update = np.logical_and((fx < fp), fs)
        p[i_update, :] = x[i_update, :].copy()
        fp[i_update] = fx[i_update]

        # Compare swarm's best position with global best position
        i_min = np.argmin(fp)
        if fp[i_min] < fg:
            if debug:
                print('New best for swarm at iteration {:}: {:} {:}'\
                    .format(it, p[i_min, :], fp[i_min]))

            p_min = p[i_min, :].copy()
            stepsize = np.sqrt(np.sum((g - p_min)**2))

            if np.abs(fg - fp[i_min]) <= minfunc:
                print('Stopping search: Swarm best objective change less than {:}'\
                    .format(minfunc))
                if particle_output:
                    return p_min, fp[i_min], p, fp
                else:
                    return p_min, fp[i_min]
            elif stepsize <= minstep:
                print('Stopping search: Swarm best position change less than {:}'\
                    .format(minstep))
                if particle_output:
                    return p_min, fp[i_min], p, fp
                else:
                    return p_min, fp[i_min]
            else:
                g = p_min.copy()
                fg = fp[i_min]

        if debug:
            print('Best after iteration {:}: {:} {:}'.format(it, g, fg))
        it += 1

    print('Stopping search: maximum iterations reached --> {:}'.format(maxiter))

    if not is_feasible(g):
        print("However, the optimization couldn't find a feasible design. Sorry")
    if particle_output:
        return g, fg, p, fp
    else:
        return g, fg



In [31]:
#         0  1            2         3  4  5  6    7  8  9     10 11 12    13   14 15 16  17 18 19   20   21 22 23 24 25 26 27 28   29   30 31 32 33 34 35 36 37
ub = [54451, 1379963888 ,309937401 ,1, 3, 3, 101, 4, 1, 7479, 1, 1, 7468, 100, 2, 9, 10, 1, 1, 511, 511, 1, 1, 1, 1, 1, 1, 1, 255, 255, 1, 1, 1, 1, 1, 1, 1, 1]

def fitness_function(X, *args):
  x_list = X.tolist()
  # Kết hớp phần bất định và phần thay đổi lại thành 1
  immutable = list(args)
  particle = x_list + immutable
  #p = np.array(particle).reshape(1, -1)

  # Chuẩn hóa dữ liệu
  p_df = pd.DataFrame(data=[particle], columns=colNames_test)
  p_df[features_integer] = p_df[features_integer].astype(int)
  p_df[features_binary] = p_df[features_binary].round().astype(int)
  test=scaler_minmax.transform(np.array(particle).reshape(1,-1))

  # Tính tỉ lệ được phân loại là lành tính
  p_proba = voting_classifier.predict_proba(test)
  return p_proba[0][1]



def AdversarialPSO(original_vector):
    np.random.seed(int(time.time()))
  # Split vecto to two parts immutable and mutable
    immutable = original_vector[dumcols].tolist()
    mutable = original_vector.drop(labels=dumcols)

    lb = mutable.tolist()

    g, fg = pso(fitness_function, lb, ub, swarmsize=200, omega=0.7, phip=0.5, phig=0.4, maxiter=5, minstep=0.001, args=immutable, debug=True)
    result = list(g) + immutable
    return result

In [35]:
# TEST
result = []
for i in range(50):
  print(f"---------row{i}-------------")
  particle_seed = malicious_vector.loc[i]
  r = AdversarialPSO(particle_seed)
  result.append(r)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
data_adv = scaler_minmax.transform(result)
result_PSO = bag_model.predict_proba(data_adv)
print(result_PSO)
print(Evaluate(result_PSO))

[[0.2 0.8]
 [0.2 0.8]
 [0.3 0.7]
 [0.3 0.7]
 [0.2 0.8]
 [0.3 0.7]
 [0.3 0.7]
 [0.  1. ]
 [0.1 0.9]
 [0.1 0.9]]
0.2


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [38]:
adv_pso_df = pd.DataFrame(result, columns=colNames_test)
adv_pso_df[features_integer] = adv_pso_df[features_integer].astype(int)
adv_pso_df[features_binary] = adv_pso_df[features_binary].astype(int)

adv_pso_df.to_csv('/content/drive/MyDrive/Model/adv_pso.csv', index=False)

In [67]:
adv_svm = svm_model.predict_proba(data_adv)
print(f"ADV PSO on SVM MODEL: {Evaluate(adv_svm)}")

adv_dt = dt_model.predict_proba(data_adv)
print(f"ADV PSO on DT MODEL: {Evaluate(adv_dt)}")

adv_nb = nb_model.predict_proba(data_adv)
print(f"ADV PSO on NB MODEL: {Evaluate(adv_nb)}")

adv_knn = knn_model.predict_proba(data_adv)
print(f"ADV PSO on KNN MODEL: {Evaluate(adv_knn)}")

adv_rf = rf_model.predict_proba(data_adv)
print(f"ADV PSO on RF MODEL: {Evaluate(adv_rf)}")

adv_mlp = mlp_model.predict_proba(data_adv)
print(f"ADV PSO on MLP MODEL: {Evaluate(adv_mlp)}")

adv_gb = gb_model.predict_proba(data_adv)
print(f"ADV PSO on GB MODEL: {Evaluate(adv_gb)}")

adv_lr = lr_model.predict_proba(data_adv)
print(f"ADV PSO on lr MODEL: {Evaluate(adv_lr)}")

adv_lda = lda_model.predict_proba(data_adv)
print(f"ADV PSO on LDA MODEL: {Evaluate(adv_lda)}")

adv_qda = qda_model.predict_proba(data_adv)
print(f"ADV PSO on QDA MODEL: {Evaluate(adv_qda)}")

adv_bag = bag_model.predict_proba(data_adv)
print(f"ADV PSO on BAG MODEL: {Evaluate(adv_bag)}")


ADV PSO on SVM MODEL: 0.9999998986253056
ADV PSO on DT MODEL: 0.8132384766356339
ADV PSO on NB MODEL: 1.0
ADV PSO on KNN MODEL: 0.8999999999999998
ADV PSO on RF MODEL: 0.8132384766356339
ADV PSO on MLP MODEL: 0.5876580764963506
ADV PSO on GB MODEL: 0.12672280656701246
ADV PSO on lr MODEL: 0.004301575911999311
ADV PSO on LDA MODEL: 0.3
ADV PSO on QDA MODEL: 0.0
ADV PSO on BAG MODEL: 0.2


In [68]:
print((Evaluate(adv_svm) + Evaluate(adv_dt) + Evaluate(adv_nb) + Evaluate(adv_knn) + Evaluate(adv_rf) + Evaluate(adv_mlp) + Evaluate(adv_gb) + Evaluate(adv_lr) + Evaluate(adv_lda) + Evaluate(adv_qda) + Evaluate(adv_bag)) / 11)

0.5222872100792668
